<a href="https://colab.research.google.com/github/salmenhsairi/EndOfStudiesProjectNotebooks/blob/main/Layoutlmv3Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount drive fs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing requirements

In [ ]:
! pip install -q git+https://github.com/huggingface/transformers.git
! pip install -q git+https://github.com/huggingface/datasets.git "dill<0.3.5" seqeval

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 25.1 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 58.4 MB/s 
     |████████████████████████████████| 346 kB 32.6 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 140 kB 71.5 MB/s 
     |████████████████████████████████| 212 kB 72.7 MB/s 
     |████████████████████████████████| 86 kB 7.8 MB/s 
     |████████████████████████████████| 1.1 MB 60.5 MB/s 
     |████████████████████████████████| 127 kB 73.9 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 144 kB 72.6 MB/s 
     |████████████████████████████████| 271 kB 71.9 MB/s 
     |████████████████████████████████| 112 kB 73.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account 

## Pulling preprocessing file

In [ ]:
! rm -r layoutlmv3FineTuning
! git clone -b master https://github.com/salmenhsairi/layoutlmv3FineTuning.git

rm: cannot remove 'layoutlmv3FineTuning': No such file or directory
Cloning into 'layoutlmv3FineTuning'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


## Loading UBIAI dataset from Drive

In [ ]:
#!/bin/bash
# IOB_DATA_PATH = "/content/drive/MyDrive/NERDatasets/OCR_Invoice_Extraction_Tp4PlPQ.zip"
IOB_DATA_PATH = "/content/drive/MyDrive/NERDatasets/EPIC_Case_Times_Extraction_ddvqdMl.zip"
! cd /content/
! rm -r data
! mkdir data
! cp "$IOB_DATA_PATH" data/dataset.zip 
! cd data && unzip -q dataset && rm dataset.zip
! cd ..

rm: cannot remove 'data': No such file or directory


## defining preprocessing params and running the script

In [ ]:
#!/bin/bash
#preprocessing args
TEST_SIZE = 0.2
DATA_OUTPUT_PATH = "/content/"

In [ ]:
! python3 layoutlmv3FineTuning/preprocess.py --valid_size $TEST_SIZE --output_path $DATA_OUTPUT_PATH

100% 1/1 [00:19<00:00, 19.76s/ba]
Downloading: 100% 275/275 [00:00<00:00, 59.0kB/s]
Downloading: 100% 1.12k/1.12k [00:00<00:00, 1.04MB/s]
Downloading: 100% 856/856 [00:00<00:00, 1.25MB/s]
Downloading: 100% 878k/878k [00:00<00:00, 54.3MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 27.1MB/s]
100% 1/1 [00:08<00:00,  8.83s/ba]
100% 1/1 [00:05<00:00,  5.65s/ba]


## Proceed with Training

In [ ]:
from datasets import load_metric
from transformers import TrainingArguments, Trainer
from transformers import LayoutLMv3ForTokenClassification,AutoProcessor
from transformers.data.data_collator import default_data_collator

In [ ]:
# load datasets
from datasets import load_from_disk
train_dataset = load_from_disk(f'/content/train_split')
eval_dataset = load_from_disk(f'/content/eval_split')

In [ ]:
labels = train_dataset.features["labels"].feature.names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [ ]:
metric = load_metric("seqeval")
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels,zero_division='0')
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

## loading model and preprocessor (also required for Hugging face trainer)

In [ ]:
label_list = train_dataset.features["labels"].feature.names
num_labels = len(label_list)
label2id, id2label = dict(), dict()
for i, label in enumerate(label_list):
    label2id[label] = i
    id2label[i] = label

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                         id2label=id2label,
                                                         label2id=label2id)

processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## let's train the model

In [ ]:
training_args = TrainingArguments(output_dir="test",
                                  # max_steps=2500,
                                  num_train_epochs=10,
                                  logging_strategy="epoch",
                                  save_total_limit=1,
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  learning_rate=4e-5,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  # eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1")

In [ ]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# trainer.train()

evaluate the model

In [ ]:
# trainer.evaluate()

## save the model for upcoming fine-tuning/infrence

In [ ]:
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3')

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3
Configuration saved in /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3/preprocessor_config.json
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3/special_tokens_map.json


## Optional Hyperparameter Tuning

### installing HPO & Experiment tracking requirements

In [ ]:
!pip install ray[tune]
!pip install wandb
!pip install pickle5==0.0.10
! pip install -U hyperopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 139 kB/s 
     |████████████████████████████████| 4.1 MB 58.4 MB/s 
     |████████████████████████████████| 8.8 MB 56.9 MB/s 
     |████████████████████████████████| 125 kB 70.6 MB/s 
     |████████████████████████████████| 461 kB 70.9 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.46.3
    Uninstalling grpcio-1.46.3:
      Successfully uninstalled grpcio-1.46.3


### proceeding with HPO

In [ ]:
import os
import wandb
import gc
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLogger
from ray.tune.schedulers import ASHAScheduler,PopulationBasedTraining,AsyncHyperBandScheduler
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune import CLIReporter
from ray.tune.integration.wandb import WandbLoggerCallback
from ray import tune
os.environ["WANDB_API_KEY"] = "d1c1ed1fca94311e0b9893ec7ae13da6de91c0f9"
os.environ["WANDB_PROJECT"]="LayoutLMV3-RayTuneAPI-PBT"
wandb.init(project=os.environ["WANDB_PROJECT"], entity="salmen_hs")

wandb: Currently logged in as: salmen_hs. Use `wandb login --relogin` to force relogin


In [ ]:
# defining tune search space
config = {
    # "learning_rate": tune.uniform(1e-5, 5e-5),
    "learning_rate": tune.choice([3e-5,4e-5, 5e-5]),
    # "warmup_ratio" : tune.grid_search([0,0.05]),
    "num_train_epochs" : tune.choice([4,5,6]),
    # "warmup_ratio" : tune.quniform(0,0.1,0.05),
    # "weight_decay" : tune.quniform(0,0.2,0.05),
    # "weight_decay" : tune.grid_search([0,0.1]),
    "per_device_train_batch_size": tune.choice([1,2]),
    "wandb": {
    "project": os.environ["WANDB_PROJECT"],
    "api_key": os.environ.get("API_KEY"),
    "log_config": True
    }
    }
# defining models evaluation metric
objective_metric = 'eval_f1'


In [ ]:
# optionally run HPO on a partition of the data
# train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10)
# Initialize our Trainer for HPO
training_args = TrainingArguments(output_dir="test",
                                  # max_steps=2500,
                                  num_train_epochs=6,
                                  logging_strategy="epoch",
                                  # save_total_limit=1,
                                  per_device_train_batch_size=2,
                                  per_device_eval_batch_size=1,
                                  learning_rate=4e-5,
                                  skip_memory_metrics=True,
                                  evaluation_strategy="epoch",
                                  save_strategy="no",
                                  fp16=True,
                                  report_to="wandb",
                                  # eval_steps=100,
                                  # load_best_model_at_end=True,
                                  # metric_for_best_model="f1"
                                 )

eval_dataset = eval_dataset.shard(index=0, num_shards=2)
trainer = Trainer(
    model_init=lambda _ : model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

Using cuda_amp half precision backend


In [ ]:
reporter = CLIReporter(
    parameter_columns={
        "learning_rate": "learning_rate",
        "num_train_epochs": "num_train_epochs",
        "per_device_train_batch_size": "per_device_train_batch_size",
    },
    metric_columns=["eval_f1"],
)

In [ ]:
# hyperopt_search = HyperOptSearch(
#     metric=objective_metric, mode="max",
#     )
# hyper_band_scheduler = AsyncHyperBandScheduler(metric=objective_metric,
#                             mode="max",max_t=6,grace_period=3)

pbt_scheduler = PopulationBasedTraining(
    time_attr = 'training_iteration',
    metric=objective_metric,
    mode="max",
    perturbation_interval=3, #The time interval T after which you perform the PBT operation
    hyperparam_mutations={ 
        "learning_rate": [3e-5,4e-5, 5e-5],
        "num_train_epochs": [3,4,5],
        "per_device_train_batch_size": [1,2]
    })

In [ ]:
best_run = trainer.hyperparameter_search(
    n_trials=4,
    direction="maximize",
    backend="ray",
    # scheduler = ASHAScheduler(
    #   max_t=6,
    #   metric = objective_metric,
    #   # metric='eval_'+metric_name,
    #   mode="max",
    #   grace_period=2,
    #   reduction_factor=2),
    scheduler = pbt_scheduler,
    # search_alg = hyperopt_search,
    resources_per_trial = {"cpu":2,'gpu':1},
    hp_space=lambda _: config,
    progress_reporter=reporter,
    keep_checkpoints_num=2,
    # loggers=DEFAULT_LOGGERS + (WandbLogger, ),
    callbacks=[
    WandbLoggerCallback(api_key_file=os.environ.get("API_KEY"), project=os.environ["WANDB_PROJECT"])
],
    log_to_file=True,
)

Currently keeping 2 checkpoints for each trial. Checkpoints are usually huge, consider setting `keep_checkpoints_num=1`.
/usr/local/lib/python3.7/dist-packages/ray/tune/tune.py:501: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
  "Consider boosting PBT performance by enabling `reuse_actors` as "
/usr/local/lib/python3.7/dist-packages/ray/tune/callback.py:291: FutureWarning: Please update `setup` method in callback `<class 'ray.tune.integration.wandb.WandbLoggerCallback'>` to match the method signature in `ray.tune.callback.Callback`.
  FutureWarning,
2022-06-10 15:06:50,025	WARNING trial_runner.py:1481 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler to access this TrialRunner API, please reach out to Ray team on GitHub. A more strict API access pattern would be enforced starting 1.12s.0


== Status ==
Current time: 2022-06-10 15:06:50 (running for 00:00:00.26)
Memory usage on this node: 4.2/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2022-06-10_15-06-49
Number of trials: 4/4 (3 PENDING, 1 RUNNING)
+------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+
| Trial name             | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |
|------------------------+----------+-----------------+-----------------+--------------------+-------------------------------|
| _objective_f4319_00000 | RUNNING  | 172.28.0.2:1155 |           5e-05 |                  4 |                             1 |
| _objective_f4319_00001 | PENDING  |                 |           5e-05 |                  4 |            

(_objective pid=1155) /usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(_objective pid=1155)   FutureWarning,
(_objective pid=1155) wandb: Currently logged in as: salmen_hs. Use `wandb login --relogin` to force relogin


== Status ==
Current time: 2022-06-10 15:07:07 (running for 00:00:17.05)
Memory usage on this node: 6.4/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2022-06-10_15-06-49
Number of trials: 4/4 (3 PENDING, 1 RUNNING)
+------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+
| Trial name             | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |
|------------------------+----------+-----------------+-----------------+--------------------+-------------------------------|
| _objective_f4319_00000 | RUNNING  | 172.28.0.2:1155 |           5e-05 |                  4 |                             1 |
| _objective_f4319_00001 | PENDING  |                 |           5e-05 |                  4 |            

(_objective pid=1155) wandb: Tracking run with wandb version 0.12.18
(_objective pid=1155) wandb: Run data is saved locally in /root/ray_results/_objective_2022-06-10_15-06-49/_objective_f4319_00000_0_learning_rate=0.0001,num_train_epochs=4,per_device_train_batch_size=1_2022-06-10_15-06-50/wandb/run-20220610_150706-2igk51zv
(_objective pid=1155) wandb: Run `wandb offline` to turn off syncing.
(_objective pid=1155) wandb: Syncing run sandy-brook-12
(_objective pid=1155) wandb: ⭐️ View project at https://wandb.ai/salmen_hs/LayoutLMV3-RayTuneAPI-PBT
(_objective pid=1155) wandb: 🚀 View run at https://wandb.ai/salmen_hs/LayoutLMV3-RayTuneAPI-PBT/runs/2igk51zv
  0%|          | 0/244 [00:00<?, ?it/s]
(_objective pid=1155) /usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:811: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
(_objective pid=1155)   "The `device` argument is deprecated and will be removed in v5 of Transformers.", 

...

(_objective pid=4848) 
 98%|█████████▊| 39/40 [00:10<00:00,  3.62it/s]
(_objective pid=4848) 
100%|██████████| 40/40 [00:11<00:00,  3.26it/s]


Result for _objective_f4319_00003:
  date: 2022-06-10_15-38-48
  done: false
  episodes_total: 0
  epoch: 6.0
  eval_accuracy: 0.9703654922621008
  eval_f1: 0.7363420427553444
  eval_loss: 0.11709780991077423
  eval_precision: 0.6458333333333334
  eval_recall: 0.856353591160221
  eval_runtime: 12.8629
  eval_samples_per_second: 3.11
  eval_steps_per_second: 3.11
  experiment_id: d1b1a9c7d1d04d6eb34d1bcc07ac33c1
  hostname: 02b4b5409ffb
  iterations_since_restore: 6
  node_ip: 172.28.0.2
  objective: 3.208894459511
  pid: 4848
  time_since_restore: 423.1941156387329
  time_this_iter_s: 62.07265377044678
  time_total_s: 576.2901437282562
  timestamp: 1654875528
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 6
  trial_id: f4319_00003
  warmup_time: 0.02201986312866211
  
(_objective pid=4848) {'eval_loss': 0.11709780991077423, 'eval_precision': 0.6458333333333334, 'eval_recall': 0.856353591160221, 'eval_f1': 0.7363420427553444, 'eval_accuracy': 0.9703654922621008,

                                                 
100%|██████████| 366/366 [06:18<00:00,  1.03s/it]


Result for _objective_f4319_00003:
  date: 2022-06-10_15-38-48
  done: true
  episodes_total: 0
  epoch: 6.0
  eval_accuracy: 0.9703654922621008
  eval_f1: 0.7363420427553444
  eval_loss: 0.11709780991077423
  eval_precision: 0.6458333333333334
  eval_recall: 0.856353591160221
  eval_runtime: 12.8629
  eval_samples_per_second: 3.11
  eval_steps_per_second: 3.11
  experiment_id: d1b1a9c7d1d04d6eb34d1bcc07ac33c1
  experiment_tag: 3_learning_rate=0.0001,num_train_epochs=6,per_device_train_batch_size=1
  hostname: 02b4b5409ffb
  iterations_since_restore: 6
  node_ip: 172.28.0.2
  objective: 3.208894459511
  pid: 4848
  time_since_restore: 423.1941156387329
  time_this_iter_s: 62.07265377044678
  time_total_s: 576.2901437282562
  timestamp: 1654875528
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 6
  trial_id: f4319_00003
  warmup_time: 0.02201986312866211
  
== Status ==
Current time: 2022-06-10 15:38:59 (running for 00:32:08.99)
Memory usage on this node: 8.8/12.

2022-06-10 15:38:59,153	INFO tune.py:748 -- Total run time: 1929.34 seconds (1929.00 seconds for the tuning loop).
